<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [22]</a>'.</span>

<img src="../images/AzPTravel_PPM.png">

## Source File Consolidation

#### This script fetches the current data collection submission files, consolidates them in to a combined csv file, and outputs that to the network.
- All source files stored in the path are read, summarized, and then consolidated into a single dataframe named 'df'.
- This process assumes that the multiple sources are in no way duplicates of each other.
- df is then output in the data collection subfolder of the "Production" folder.


### User Variables
- These are overwritten if inherited from run_control.ipynb.
- Feel Free to reset them for a manual run if you like
- Do not save without percode = "-f"

In [1]:
commit_message = "Development and testing."
# Give a brief reason for the run.

run_control = 1
#run_type = 0 - Lite run with no reporting, not recommended.
#run_type = 1 - Lite run with normal reporting, default setting.
#run_type = 2 - Heavy run with full reporting, available for audits and troubleshooting.
#run_type = 5 - A default setting. Indicates the script is being run by an outside process without an inherited value

percode = "2021.Q1"
# Data Collection Code, this controls file paths and output names
# "-f" is the value indicating a bad inheritance from run with arg

s_format = "p"
# denotes the source data format x == Excel; j == json, p == parquet

#----------
# do not edit - this either inherits the full instance timestamp from the papermill book or captures the run time of this script.
from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")

In [2]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Live run, add CH dist Channels"
inst_datetime = "06152021135007"
source_type = "p"


#### Notebook display options

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### import packages

In [4]:
#### Packages used

import os # System commands
import sys # System commands

from datetime import datetime # datetime options
import warnings # custom warnigns options

import getpass # parquet file read/write
import json # json file read/write

import matplotlib.pyplot as plt #Plots and Graphs
import openpyxl # Excel operations
import numpy as np
import pandas as pd #DataFrame and math





#### Default Variables, these govern logic, do not edit.

In [5]:
default_dc = "20XX.QX"
default_rc = 0 #extra lite mode
dummy_perc = "33Q3" # bad inheritance

#### Script determining run context ie, manual, run_control.ipynb, or other.

In [6]:
if run_control == 5:
    run_control = default_rc 
else:
    run_control = run_control

try:
    if sys.argv[1] == "-f":
        percode = percode
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_dc
except NameError:
    percode = default_dc


#### Make paths for the source folder

In [7]:
rt_path = f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}'
ls_path = os.path.join( rt_path, 'live_sources')


#### Get a list of only source files in the path that start with "us_dat".
#### Logic determines the source file types.

#### User instructions:
- Make sure that you have 1 file per source in this folder.
    -  For instance, do not have two files for Portugal. If there is an update, archive the old one.
- Do not overwrite files in the archive.
    - Rename newly archived files, no strict convention, we keep track of these by the modified date.
- It is ok to have multiple sources in one file.

In [8]:
files = os.listdir(ls_path)
files = [files.lower() for files in files]

if s_format == "x":
    files_sour = [f for f in files if f[-3:] in ('lsx' , 'lsm' ,'xls')]

elif s_format == "j":
    files_sour = [f for f in files if f[:6] == 'us_dat'  and  f[-5:]  == '.json']

elif s_format == "p":
    files_sour = [f for f in files if f[:6] == 'us_dat' and  f[-8:]  == '.parquet']


#### Create a list 'pathfiles' that has every source file with the full path.

In [9]:
pathfiles = []

for f in files_sour:
    makepathsfiles = os.path.join(str(ls_path), str(f))
    pathfiles.append(makepathsfiles)


#### File Check

- create file information Data.Frame "file_info_df" with name, size, and modified date of each source file
- import previous source file list for comparison
- print all new files, store as a list "newfiles"
- print all dropped files, store as a list "dropped_files"

In [10]:
#print("Current Source Files")
cols = ["Filename", "Size", "Last Modified" ]

file_info_df = pd.DataFrame(columns = cols)
for f, p in zip(files_sour, pathfiles):
    name = f
    size = os.path.getsize(p) # size in bytes
    moddate = os.path.getctime(p)# time of last metadata change;
    moddate =  datetime.fromtimestamp(moddate).strftime('%Y-%m-%d %H:%M:%S')# format change;
    new_row = pd.DataFrame([[name, size, moddate]], columns = cols)
    file_info_df = file_info_df.append(new_row, ignore_index=True)

#print(file_info_df)

file_list = os.path.join( rt_path, 'Current_Source_Files.csv')
prev_files = pd.read_csv(file_list , encoding='utf-8', engine='python')


fl_comp = file_info_df["Filename"] + file_info_df["Size"].astype(str) + file_info_df["Last Modified"].astype(str)

prev_comp = prev_files["Filename"] + prev_files["Size"].astype(str) + prev_files["Last Modified"].astype(str)

newfiles = []

for idx, r in enumerate(fl_comp):
    
    if r in prev_comp.values:
    
        pass
     
    else:
        if file_info_df['Filename'][idx] in prev_files["Filename"].values:
            print("")
            print(f"Changed data file found: {file_info_df['Filename'][idx]}, last update {file_info_df['Last Modified'][idx]}.")
            newfiles.append(pathfiles[idx])
        else:
            print("")
            print(f"New data file found: {file_info_df['Filename'][idx]}, last update {file_info_df['Last Modified'][idx]}.")
            newfiles.append(pathfiles[idx])

if len(newfiles) == 0:
    
    print("No new or changed data files found in live sources folder.")

else:

    pass

dropped_files = []

for r in prev_files["Filename"]:
    
    if r in file_info_df["Filename"].values:
        
        pass
    
    else:
        print("")
        print(f"File {r} has been archived or removed from live sources folder.")
        dropped_files.append(os.path.join( ls_path, r))
    
    
    
    
file_info_df.to_csv(file_list, index = False)





New data file found: us_dat_de_06082021183745.parquet, last update 2021-06-15 13:48:32.

File us_dat_de_06082021183744.parquet has been archived or removed from live sources folder.


#### Business Unit Check

- import existing Business Unit and raw file list
- enusure there is a 1 to 1 relationship
- throw error if not


In [11]:
cols = [ "Business Unit", "File"]

bu_x_file = pd.DataFrame(columns = cols ) 

for p, f  in zip(pathfiles,files_sour):

    if s_format == "x":

        try:
            data = pd.read_excel(p, sheet_name = 'Portfolio_Monitoring', na_values = [0], header=3, converters={ 'Business Partner Name': str, 'Type of Business': str, 'Type of Account': str, 'Distribution Type': str, 'LOB': str, 'Distribution Channel': str,
            'Sub LOB': str,'Business Partner ID Number': str,  'Product Name': str, 'Product ID Number': str,  'Product Family': str,  'Standard Product': str, 'Total Expenses': float,    })

        except:
            data = pd.read_excel(p, sheet_name = 'Ptf_Monitoring_GROSS_Reins', na_values = [0], header=3, converters={ 'Business Partner Name': str, 'Type of Business': str, 'Type of Account': str, 'Distribution Type': str, 'LOB': str, 'Distribution Channel': str,
            'Sub LOB': str,'Business Partner ID Number': str,  'Product Name': str, 'Product ID Number': str,  'Product Family': str,  'Standard Product': str, 'Total Expenses': float,    })


    elif s_format == "j":

        data = pd.read_json(p, orient="table")

    elif s_format == "p":

        data = pd.read_parquet(p, engine = "pyarrow")

    bus = data["Business Unit"].unique()
    
    for i in bus: bu_x_file = bu_x_file.append({"Business Unit": i,"File": f} , ignore_index=True)

        
#This is a list of BU's that are included in the raw data files, and the file name(s) they are contained in.
bu_filelist = bu_x_file.sort_values(by='Business Unit').style.hide_index()




In [12]:

bucount =  bu_x_file.groupby("Business Unit").count().sort_values(by='File', ascending=False)

filecount =  bu_x_file.groupby("File").count().sort_values(by='File')

err_check = 1

if len(bucount[bucount["File"] > 1]) > 0: 
    warnings.warn("\n \nThere is a Business Unit with data in multiple source files.\nReview the file counts and filenames to resolve the issue.\nThis is a critical control.")
    err_check = 0
else:
    pass

try:
    1 / err_check

except ZeroDivisionError:
    print("Execution halted due to multiple source file per one or more Business Units.")
    1 / err_check


1.0

In [13]:

print("This is a count of how many BUs appear in each file. It is fine to submit multiple BUs in one source.")
filecount


print("\nThis is a count of how many files each BU appears in. In all cases the count values should be one (1). The process as desgined, should have a single submission and source for each BU.")
bucount

print("This is a list of BU's that are included in the raw data files, and the file name(s) they are contained in.")   

bu_filelist

This is a count of how many BUs appear in each file. It is fine to submit multiple BUs in one source.


,Business Unit
File,
us_dat_at_+te_q1_2021_at.parquet,1
us_dat_au_05142021143612.parquet,1
us_dat_ca_05202021134242.parquet,1
us_dat_ch_06042021132401.parquet,1
us_dat_cz_05262021090243.parquet,1
us_dat_de_06082021183745.parquet,1
us_dat_es_05202021105141.parquet,1
us_dat_gr_05252021115404.parquet,1
us_dat_ie_06042021151138.parquet,1



This is a count of how many files each BU appears in. In all cases the count values should be one (1). The process as desgined, should have a single submission and source for each BU.


,File
Business Unit,
AT,1
AU,1
CA,1
CH,1
CZ,1
DE,1
ES,1
GR,1
IE,1


This is a list of BU's that are included in the raw data files, and the file name(s) they are contained in.


Business Unit,File
AT,us_dat_at_+te_q1_2021_at.parquet
AU,us_dat_au_05142021143612.parquet
CA,us_dat_ca_05202021134242.parquet
CH,us_dat_ch_06042021132401.parquet
CZ,us_dat_cz_05262021090243.parquet
DE,us_dat_de_06082021183745.parquet
ES,us_dat_es_05202021105141.parquet
GR,us_dat_gr_05252021115404.parquet
IE,us_dat_ie_06042021151138.parquet
IT,us_dat_it_05242021111520.parquet


- Read each json file into temp DataFrame "data".
- Prep actions - Strip (trim) leading and trailing spaces values in string columns, remove rows without business units.
- Append each json output to into 1 DataFrame "df".
- Process Cleanup, the indices restart with each append, reset and drop the previous.

In [14]:
df_dict = {}
#empty dict to append each file's DataFrame to

for f in pathfiles:

    if s_format == "x":

        try:
            data = pd.read_excel(f, sheet_name = 'Portfolio_Monitoring', na_values = [0], header=3, converters={ 'Business Partner Name': str, 'Type of Business': str, 'Type of Account': str, 'Distribution Type': str, 'LOB': str, 'Distribution Channel': str,
            'Sub LOB': str,'Business Partner ID Number': str,  'Product Name': str, 'Product ID Number': str,  'Product Family': str,  'Standard Product': str, 'Total Expenses': float,    })

        except:
            data = pd.read_excel(f, sheet_name = 'Ptf_Monitoring_GROSS_Reins', na_values = [0], header=3, converters={ 'Business Partner Name': str, 'Type of Business': str, 'Type of Account': str, 'Distribution Type': str, 'LOB': str, 'Distribution Channel': str,
            'Sub LOB': str,'Business Partner ID Number': str,  'Product Name': str, 'Product ID Number': str,  'Product Family': str,  'Standard Product': str, 'Total Expenses': float,    })


    elif s_format == "j":

        data = pd.read_json(f, orient="table")

    elif s_format == "p":

        data = pd.read_parquet(f, engine = "pyarrow")

    df_dict.update({f: data})

## Raw file Summaries

#### First 5 rows of each file.

In [15]:
if run_control > 0:

    for k, v in df_dict.items():
        print(f"{k} - First 5 Samples:")
        print("-" * 100)
        print(v.head())

else:
    print("Default Report 2 Skipped")

//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_at_+te_q1_2021_at.parquet - First 5 Samples:
----------------------------------------------------------------------------------------------------
  Business Unit Country Currency                           Region  \
0            AT      AT      EUR  North, Central & Eastern Europe   
1            AT      AT      EUR  North, Central & Eastern Europe   
2            AT      AT      EUR  North, Central & Eastern Europe   
3            AT      AT      EUR  North, Central & Eastern Europe   
4            AT      AT      EUR  North, Central & Eastern Europe   

  Reporting Date From Reporting Date To Date of Analysis Type of Analysis  \
0          2021-01-01        2021-03-31       2021-05-07     Year To Date   
1          2021-01-01        2021-03-31       2021-05-07     Year To Date   
2          2021-01-01        2021-03-31       2021-05-07     Year To Da

  Business Unit Country Currency                           Region  \
0            CZ      CZ      EUR  North, Central & Eastern Europe   
1            CZ      CZ      EUR  North, Central & Eastern Europe   
2            CZ      CZ      EUR  North, Central & Eastern Europe   
3            CZ      CZ      EUR  North, Central & Eastern Europe   
4            CZ      CZ      EUR  North, Central & Eastern Europe   

  Reporting Date From Reporting Date To Date of Analysis Type of Analysis  \
0          2021-01-01        2021-03-31       2021-05-20     Year To Date   
1          2021-01-01        2021-03-31       2021-05-20     Year To Date   
2          2021-01-01        2021-03-31       2021-05-20     Year To Date   
3          2021-01-01        2021-03-31       2021-05-20     Year To Date   
4          2021-01-01        2021-03-31       2021-05-20     Year To Date   

   Analysed Months  Reporting Year  ...  Notes  Unnamed: 66 concat2  \
0                3            2021  ...    0.0     

  Business Unit Country Currency                       Region  \
0            IE      IE      EUR  Western Europe, LATAM & MEA   
1            IE      IE      EUR  Western Europe, LATAM & MEA   
2            IE      IE      EUR  Western Europe, LATAM & MEA   
3            IE      IE      EUR  Western Europe, LATAM & MEA   
4            IE      IE      EUR  Western Europe, LATAM & MEA   

  Reporting Date From Reporting Date To Date of Analysis Type of Analysis  \
0          2021-01-01        2021-03-31       2021-06-04     Year To Date   
1          2021-01-01        2021-03-31       2021-06-04     Year To Date   
2          2021-01-01        2021-03-31       2021-06-04     Year To Date   
3          2021-01-01        2021-03-31       2021-06-04     Year To Date   
4          2021-01-01        2021-03-31       2021-06-04     Year To Date   

   Analysed Months  Reporting Year  ...                         Notes  \
0                3            2021  ...                             0   


  Business Unit Country Currency                       Region  \
0            PT      PT      EUR  Western Europe, LATAM & MEA   
1            PT      PT      EUR  Western Europe, LATAM & MEA   
2            PT      PT      EUR  Western Europe, LATAM & MEA   
3            PT      PT      EUR  Western Europe, LATAM & MEA   
4            PT      PT      EUR  Western Europe, LATAM & MEA   

  Reporting Date From Reporting Date To Date of Analysis Type of Analysis  \
0          2021-01-01        2021-03-31       2021-03-31     Year To Date   
1          2021-01-01        2021-03-31       2021-03-31     Year To Date   
2          2021-01-01        2021-03-31       2021-03-31     Year To Date   
3          2021-01-01        2021-03-31       2021-03-31     Year To Date   
4          2021-01-01        2021-03-31       2021-03-31     Year To Date   

   Analysed Months  Reporting Year  ...  \
0              3.0          2021.0  ...   
1              3.0          2021.0  ...   
2              3.

#### Summary Statisitics.

In [16]:
if run_control > 0:

    for k, v in df_dict.items():
        print(f"{k} - Statistical summary:")
        print("-" * 100)
        print(v.describe())
else:
    print("Default Report 3 Skipped")

//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_at_+te_q1_2021_at.parquet - Statistical summary:
----------------------------------------------------------------------------------------------------
       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count            222.0           222.0              222.0            222.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Dis


----------------------------------------------------------------------------------------------------
       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count             53.0            53.0               53.0             53.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Policies (Written)  Written Revenues net of Taxes  \
count                  5.300000e+01                      53.000000   
mean                   5.487073e+04                    

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count             17.0            17.0               17.0             17.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Products per Row  Number of B-Partners per Row  \
count                        17.0                          17.0   
mean                          0.0                           0.0   
std                           0.0                           0.0   
min                          

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count              7.0             7.0                7.0              7.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Products per Row  Number of B-Partners per Row  \
count                    7.000000                           7.0   
mean                     2.285714                           1.0   
std                      2.563480                           0.0   
min                      1.00

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count             28.0            28.0               28.0             28.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Products per Row  Number of B-Partners per Row  \
count                   28.000000                          28.0   
mean                     2.428571                           1.0   
std                      3.084352                           0.0   
min                      1.00

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count             14.0            14.0               14.0             14.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Products per Row  Number of B-Partners per Row  \
count                   14.000000                          14.0   
mean                     4.000000                           0.0   
std                      5.830952                           0.0   
min                      1.00

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count            151.0           151.0              151.0            151.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Number of Products per Row  Number of B-Partners per Row  \
count                       151.0                    151.000000   
mean                          1.0                     19.337748   
std                           0.0                     67.161386   
min                          

       Analysed Months  Reporting Year  Reporting Quarter  Reporting Month  \
count             40.0            40.0               40.0             40.0   
mean               3.0          2021.0                1.0              3.0   
std                0.0             0.0                0.0              0.0   
min                3.0          2021.0                1.0              3.0   
25%                3.0          2021.0                1.0              3.0   
50%                3.0          2021.0                1.0              3.0   
75%                3.0          2021.0                1.0              3.0   
max                3.0          2021.0                1.0              3.0   

       Product ID Number  Number of Products per Row  \
count               40.0                        40.0   
mean                 0.0                         1.0   
std                  0.0                         0.0   
min                  0.0                         1.0   
25%              

#### Null Value Report.

In [17]:
if run_control > 0:

    for k, v in df_dict.items():
        print(f"{k} - Null values in the dataset:")
        print("-" * 100)
        print(v.isnull().sum())
else:
    print("Default Report 4 Skipped")

//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_at_+te_q1_2021_at.parquet - Null values in the dataset:
----------------------------------------------------------------------------------------------------
Business Unit          0
Country                0
Currency               0
Region                 0
Reporting Date From    0
                      ..
Commission Ratio       0
Expense Ratio          0
Combined Ratio         0
Profit or Loss         0
Notes                  0
Length: 66, dtype: int64
//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_au_05142021143612.parquet - Null values in the dataset:
----------------------------------------------------------------------------------------------------
Business Unit                     0
Country                           0
Currency                          0
Regi

#### Datatypes by field for each file.

In [18]:
if run_control > 0:

    for k, v in df_dict.items():
        print(f"{k} - Datatypes:")
        print("-" * 100)
        print(v.dtypes)
else:
    print("Default Report 5 Skipped")

//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_at_+te_q1_2021_at.parquet - Datatypes:
----------------------------------------------------------------------------------------------------
Business Unit                  object
Country                        object
Currency                       object
Region                         object
Reporting Date From    datetime64[ns]
                            ...      
Commission Ratio              float64
Expense Ratio                 float64
Combined Ratio                float64
Profit or Loss                float64
Notes                          object
Length: 66, dtype: object
//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/2021.Q1\live_sources\us_dat_au_05142021143612.parquet - Datatypes:
----------------------------------------------------------------------------------------------------
Bu

#### Add the datafile name as a column in each source DataFrame

In [19]:
for p, f in zip(pathfiles, files_sour):
    df_dict[p].insert(0, "Submission File", f)

#### append into a single DataFrame

In [20]:
df = pd.DataFrame()

for k, v in df_dict.items(): df = df.append(v)

df = df.reset_index(drop=True)

#### Create Temporary csv Output
- Create a direct output of the csv

##### This prep step helps parquet columns read columns with nulls or mixed dtypes. This is a good check that the data is what we expect.

In [21]:

# add any float columns that give a mixed type error below
v_flo = ['Open Claims %', 'Contribution Margin % on Earned Revenues net of Taxes - BU View', 
         'Contribution Margin % on Earned Revenues net of Taxes - HQ View', 'Loss Ratio', 'Commission Ratio',
         'Expense Ratio', 'comsub', 'expsub','% of IBNR on (OCR + IBNR)', 'Contribution Margin % on Fixed Costs - BU View',
        'Contribution Margin % on Fixed Costs - HQ View', 'Units of Risk (Written)', 'Written Revenues', 
         'Number of Policies (Earned)','Units of Risk (Earned)', 'Earned Revenues','Upfront Cash Payments',
         'Number of Open Claims','Number of Persons Involved in Claims (Paid + OCR + IBNR)','Frequency (Earned)',
         'Units of Risk (Written)' ,'Severity','Risk Premium','Unnamed: 66']


# add any integer columns that give a mixed type error below
v_int = ['Number of Products per Row',  'Number of B-Partners per Row']

#v_obj = ['Units of Risk (Written)']


# add any string columns that give a mixed type error below
v_str = [ 'Distribution Type','Distribution Channel','Sub LOB', 'Business Partner ID Number', 'Product ID Number', 'Standard Product' ,'Notes']


# add any datetime columns that give a mixed type error below
#v_dat

for i in v_flo:
    df[i] = df[i].replace(r'^\s*$', np.nan, regex=True)
    df[i] = df[i].fillna(0)
    df[i] = df[i].astype( 'float' )
for i in v_int:
    df[i] = df[i].replace(r'^\s*$', np.nan, regex=True)
    df[i] = df[i].fillna(0)
    df[i] = df[i].astype('int64')

for i in v_str:
    df[i] = df[i].astype('str')
    for i in v_str: df[i] = df[i].astype('str'  )

#for i in v_dat: v[i] = v[i].astype('datetime64[ns]')

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [22]:
try:
    prep_df = df
    %store prep_df

except:
    pass

if run_control > 0:
    
    prepfile = f'{str(percode)}prep.csv'
    prepparq = f'{str(percode)}prep.parquet'
    prephistfile = f'{str(percode)}_{inst_datetime}prep.csv'

    try:
        os.rename(os.path.join(str(rt_path), prepfile), os.path.join(str(rt_path), "logs/raw_data_file_history/" , prephistfile))

    except FileNotFoundError:
        pass

    except FileExistsError:
        os.remove(os.path.join(str(rt_path), "logs/raw_data_file_history/" , prephistfile))
        os.rename(os.path.join(str(rt_path), prepfile), os.path.join(str(rt_path), "logs/raw_data_file_history/" , prephistfile))

    df.to_csv(os.path.join(str(rt_path), prepfile), ',', index=False , encoding='utf-8-sig')
    df.to_parquet(os.path.join(str(rt_path), prepparq),engine = "pyarrow")
    
else:
    pass

Stored 'prep_df' (DataFrame)


ArrowTypeError: ("Expected a bytes object, got a 'float' object", 'Conversion failed for column Product Name with type object')

## This is considered "pre-pipleine" data prep. The next step is to read the resulting csv into the acutal data pipeline.

## If you are not sure what is being loaded, this step provides a safe way to create a reviewable output file while keeping the production data files in tact.

#### You can skip this step to speed up the procoess with run_contol == 0, not reccomended

In [ ]:
if run_control > 0:

    dfp = (df.pivot_table(index=( "Country", "Submission File"), columns=("Reporting Date From","Reporting Date To" ), values="Earned Revenues net of Taxes").fillna(0).astype(int))
    dfp

else:
    pass

#### Store the DataFrame for other noteboks to use